<p align="center">
    <img src="./images/logo_beap.png">
</p>

### Renato Profeta <br> Guitars.AI <br> PhD Candidate / Researcher



# Audio CODEC Configuration

In [1]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/e1FpS2C0aYI" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

 - Different audio sources have different electrical output characteristics. <br>
 
<figure>
    <center>
        <img src="./images/03_01_level-ranges.jpg" width="700">
        <cite>https://sessionville.com/articles/signals-from-source-to-daw-part1</cite>
    </center>
</figure>


### Audio CODEC Block Diagram

<figure>
    <center>
        <img src="./images/03_05_codec_diagram.jpg" width="800">
        <cite>AIC3204 Datasheet SLOS602A</cite>
    </center>
</figure>

## ADC Gain Setting

In [3]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/fIXRhBtkC8A" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

 - "The AIC3204 features a Analog Programmable Gain Amplifier (PGA) for boosting low-level signals, such as direct
    microphone inputs, to full-scale to achieve high SNR." 

<br>
<figure>
    <center>
        <img src="./images/03_02_pga.jpg" width="800">
        <cite>AIC3204 Datasheet SLOS602A</cite>
    </center>
</figure>


 - "This gain can be user controlled by writing to Page 1, Register 59 and Page 1, Register 60."
 
 
<br>
<figure>
    <center>
        <img src="./images/03_02_pga_volume.jpg" width="800">
        <cite>AIC3204 Datasheet SLOS602A</cite>
    </center>
</figure>

## Sampling Rate Configuration

In [4]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/AyEXZ22Yguc" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

 - The AIC3204 uses a fixed oscillator on the outside (12 MHz) conected at the  MCLK pin.
 
 <br>
<figure>
    <center>
        <img src="./images/03_03_codec_mclk.jpg" width="800">
        <cite>TMS320C5515 eZdsp USB Stick Technical Reference</cite>
    </center>
</figure>

### Phase-locked Loop  (PLL)

 - A control system that can generate an accurate output signal of frequency equal to, or a multiple of, the input signal    frequency.
 - "The TLV320AIC3204 provides the option of using the on-chip PLL which supports a wide range of fractional multiplication values to generate the required clocks. Starting from CODEC_CLKIN the TLV320AIC3204 provides several programmable clock dividers to help achieve a variety of sampling rates for ADC, DAC and clocks for the processing block."
 - "The clocks for ADC and DA require a source reference clock. This clock can be provided on variety of device pins such as MCLK, BCLK or GPI pins."

<figure>
    <center>
        <img src="./images/03_06_pll_clock.jpg" width="600">
        <cite>AIC3204 Datasheet SLOS602A</cite>
    </center>
</figure>

 - The source reference clock for the codec can be chosen by programming the CODEC_CLKIN value on Page 0, Register 4, D(1:0).

 - The PLL can be programmed via Page 0, Registers 5 thru 8. 
    - The PLL can be turned on via Page 0, Register 5, D(7). 
    - The variable P can be programmed via Page 0, Register 5, D(6:4). The default register value for P is 2. 
    - The variable R can be programmed via Page 0, Register 5, D(3:0). The default register value for R is 1. 
    - The variable J can be programmed via Page 0, Register 6, D(5:0). 
    - The variable D is 12-bits and is programmed into two registers. The MSB portion can be programmed via Page 0, Register 7,      D(5:0), and the LSB portion is programmed via Page 0, Register 8, D(5:0). The default register value for D is 0.

In [5]:
def calc_PLL_CLK(PLL_CLKIN, R, P, J, D):
    return (PLL_CLKIN * R * (J+D/10**(len(str(D)))))/P

In [6]:
PLL_CLK=calc_PLL_CLK(12e6,1,1,7,1680)
PLL_CLK

86016000.0

<figure>
    <center> CODEC CLKIN Clock Dividers
        <img src="./images/03_04_clock_divider.jpg" width="800">
        <cite>AIC3204 Datasheet SLOS602A</cite>
    </center>
</figure>

In [7]:
def calc_ADC_FS(CODEC_CLKIN, NADC, MADC, AOSR):
    return (CODEC_CLKIN/(NADC*MADC*AOSR))

In [8]:
ADC_FS = calc_ADC_FS(PLL_CLK,7,2,128)
ADC_FS

48000.0

In [9]:
def calc_PLLPR(fs , NADC=7, MADC=2, AOSR=128, PLL_CLKIN = 12e6, J=7, D = 1680, R = 1):
    P = ((J+D/10**(len(str(D)))) * PLL_CLKIN * R) / (fs * (NADC * MADC * AOSR))
    return hex(128 + (int(P) << 4) + R)

In [10]:
calc_PLLPR(48000)

'0x91'

### C Function to Set Sampling Frequency and Gain

In [11]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/KWwquyBNRR8" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

aic3204.c
Function to set register:
```c
Int16 AIC3204_rset( Uint16 regnum, Uint16 regval )
{
    Uint8 cmd[2];
    cmd[0] = regnum & 0x007F;       // 7-bit Device Address
    cmd[1] = regval;                // 8-bit Register Data

    return USBSTK5505_I2C_write( AIC3204_I2C_ADDR, cmd, 2 );
}```

aic3204_init.c
Function to set Sampling Frequency and Gain
```c
unsigned long set_sampling_frequency_and_gain(unsigned long SamplingFrequency, unsigned int ADCgain)
{
    unsigned int PLLPR = 0x91; // Default to 48000 Hz 	
    unsigned int gain;
    unsigned long output;

    if ( ADCgain >= 48)
     {
      gain = 95;      //  Limit gain to 47.5 dB
      ADCgain = 48;   // For display using printf()
     }
    else 
    {
     gain = (ADCgain << 1); // Convert 1dB steps to 0.5dB steps
    }
     
    switch (SamplingFrequency)
    {
     case 48000:
     	PLLPR = 0x91; // 1001 0001b. PLL on. P = 1, R = 1. 
     	printf("Sampling frequency 48000 Hz Gain = %2d dB\n", ADCgain);
     	output = 48000;
     break;
     
     case 24000:
      	PLLPR = 0xA1; // 1010 0001b. PLL on. P = 2, R = 1.
      	printf("Sampling frequency 24000 Hz Gain = %2d dB\n", ADCgain);
      	output = 24000;   
     break;
     
     case 16000:
     	PLLPR = 0xB1; // 1011 0001b. PLL on. P = 3, R = 1.
      	printf("Sampling frequency 16000 Hz Gain = %2d dB\n", ADCgain); 
      	output = 16000;      	
     break;
     
     case 12000:
        PLLPR = 0xC1; //1100 0001b. PLL on. P = 4, R = 1.
        printf("Sampling frequency 12000 Hz Gain = %2d dB\n", ADCgain);
        output = 12000;  
     break;
     
     case 9600:
     	PLLPR = 0xD1; //1101 0001b. PLL on. P = 5, R = 1.
       	printf("Sampling frequency 9600 Hz Gain = %2d dB\n", ADCgain); 
       	output = 9600; 
     break;
     
     case 8000:
     	PLLPR = 0xE1; //1110 0001b. PLL on. P = 6, R = 1.
     	printf("Sampling frequency 8000 Hz Gain = %2d dB\n", ADCgain);
     	output = 8000;  
     break;   	
     
     case 6857:
     	PLLPR = 0xF1; //1111 0001b. PLL on. P = 7, R = 1.
     	printf("Sampling frequency 6857 Hz Gain = %2d dB\n", ADCgain);  
     	output = 6857;    
     break;
     
     default:
     	PLLPR = 0x91; // 1001 0001b. PLL on. P = 1, R = 1. 
     	printf("Sampling frequency not recognised. Default to 48000 Hz Gain = %2d dB\n", ADCgain);
     	output = 48000;  
     break;
    } 
 

     
    /* Configure Serial Bus */
    SYS_EXBUSSEL |= 0x0100;  // Configure Serial bus 0 for I2S0
 
     
    /* Configure AIC3204 */

    AIC3204_rset( 0, 0 );      // Select page 0
    AIC3204_rset( 1, 1 );      // Reset codec
    AIC3204_rset( 0, 1 );      // Point to page 1
    AIC3204_rset( 1, 8 );      // Disable crude AVDD generation from DVDD
    AIC3204_rset( 2, 1 );      // Enable Analog Blocks, use LDO power
    AIC3204_rset( 0, 0 );
    /* PLL and Clocks config and Power Up  */
    AIC3204_rset( 27, 0x1d );  // BCLK and WCLK is set as o/p to AIC3204(Master)
    AIC3204_rset( 28, 0x00 );  // Data ofset = 0
    AIC3204_rset( 4, 3 );      // PLL setting: PLLCLK <- MCLK, CODEC_CLKIN <-PLL CLK
    AIC3204_rset( 6, 7 );      // PLL setting: J=7
    AIC3204_rset( 7, 0x06 );   // PLL setting: HI_BYTE(D)
    AIC3204_rset( 8, 0x90 );   // PLL setting: LO_BYTE(D)
    AIC3204_rset( 30, 0x88 );  // For 32 bit clocks per frame in Master mode ONLY
                               // BCLK=DAC_CLK/N =(12288000/8) = 1.536MHz = 32*fs
    AIC3204_rset( 5, PLLPR );   //PLL setting: Power up PLL, P=1 and R=1
    AIC3204_rset( 13, 0 );     // Hi_Byte(DOSR) for DOSR = 128 decimal or 0x0080 DAC oversamppling
    AIC3204_rset( 14, 0x80 );  // Lo_Byte(DOSR) for DOSR = 128 decimal or 0x0080
    AIC3204_rset( 20, 0x80 );  // AOSR for AOSR = 128 decimal or 0x0080 for decimation filters 1 to 6
    AIC3204_rset( 11, 0x87 );  // Power up NDAC and set NDAC value to 7
    AIC3204_rset( 12, 0x82 );  // Power up MDAC and set MDAC value to 2
    AIC3204_rset( 18, 0x87 );  // Power up NADC and set NADC value to 7
    AIC3204_rset( 19, 0x82 );  // Power up MADC and set MADC value to 2
    /* DAC ROUTING and Power Up */
    AIC3204_rset( 0, 1 );      // Select page 1
    AIC3204_rset( 0x0c, 8 );   // LDAC AFIR routed to HPL
    AIC3204_rset( 0x0d, 8 );   // RDAC AFIR routed to HPR
    AIC3204_rset( 0, 0 );      // Select page 0
    AIC3204_rset( 64, 2 );     // Left vol=right vol
    AIC3204_rset( 65, 0 );     // Left DAC gain to 0dB VOL; Right tracks Left
    AIC3204_rset( 63, 0xd4 );  // Power up left,right data paths and set channel
    AIC3204_rset( 0, 1 );      // Select page 1
    AIC3204_rset( 0x10, 10 );  // Unmute HPL , 10dB gain
    AIC3204_rset( 0x11, 10 );  // Unmute HPR , 10dB gain
    AIC3204_rset( 9, 0x30 );   // Power up HPL,HPR
    AIC3204_rset( 0, 0 );      // Select page 0
    USBSTK5505_wait( 100 );    // wait
    /* ADC ROUTING and Power Up */
    AIC3204_rset( 0, 1 );      // Select page 1
    AIC3204_rset( 0x34, 0x10 );// STEREO 1 Jack
		                       // IN2_L to LADC_P through 0 kohm
    AIC3204_rset( 0x37, 0x10 );// IN2_R to RADC_P through 0 kohmm
    AIC3204_rset( 0x36, 1 );   // CM_1 (common mode) to LADC_M through 0 kohm
    AIC3204_rset( 0x39, 0x40 );// CM_1 (common mode) to RADC_M through 0 kohm
    AIC3204_rset( 0x3b, gain );   // MIC_PGA_L unmute
    AIC3204_rset( 0x3c, gain );   // MIC_PGA_R unmute
    AIC3204_rset( 0, 0 );      // Select page 0
    AIC3204_rset( 0x51, 0xc0 );// Powerup Left and Right ADC
    AIC3204_rset( 0x52, 0 );   // Unmute Left and Right ADC
    
    AIC3204_rset( 0, 0 );    
    USBSTK5505_wait( 100 );  // Wait
    
    /* I2S settings */
    I2S0_SRGR = 0x0;     
    I2S0_CR = 0x8010;    // 16-bit word, slave, enable I2C
    I2S0_ICMR = 0x3f;    // Enable interrupts

 	return(output);
}
```

## Configuring Sampling Rate and Input Gain: Code Composer Studio and Board

In [12]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/KWwquyBNRR8" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>